# Invariantin massan laskeminen

Tässä esimerkissä opetellaan laskemaan __invariantti massa__ CMS:n avoimen datan arvojen avulla. Invariantti massa on hiukkasfyysikoille tärkeä työkalu hiukkasten tunnistamisessa.



## Invariantti massa?
Hiukkasilla on nille ominainen massa joka kuvaa hiukkasen hitautta. Se on sama massa $m$ jota käytetään Newtonin toisessa laissa $F=ma$ ja Einsteinin yhtälössä $E=mc^2$ (missä E viittaa hiukkasen energiaan levossa). On mahdoton ajatus punnita esimerkiksi Higgsin hiukkasta jonka eliniänodote on luokkaa $10^{-22}$ sekunttia [4]. Hiukkasen hajoamisesta sinkoilevat uudet hiukkaset taas saattavat kulkea jopa valonnopeudella eikä niillä välttämättä edes ole massaa (esim. photonit). CMS:n kaltainen laite voi kuitenkin mitata tytärhiukkasten liikemäärää ja energiaa, niiden avulla voidaan laskea _invariantti massa_. Se on matemaattinen arvo joka on "invariantti" mittausympäristölle, eli se voidaan laskea missä ja mille tahanas. Jos sen kuitenkin laskee tietystä hajoamisesta syntyneille tytärhiukkasille saadaan arvo joka on lähellä emohiukkasen massaa.

Invariantti massa siis säilyy hiukkasen hajotessa uusiksi hiukkasiksi. Tämän ominaisuuden ansiosta se on voittamaton työkalu kokeellisen hiukkasfysiikan tutkijoille. Hiukkasilmaisin, kuten CMS, havaitsee lähinnä stabiileja hiukkasia. Määrittääkseen minkä raskaamman hiukkasen hajoamisesta stabiilit hiukkaset ovat kotoisin, tutkijoiden tulee tarkastella suuria datamääriä. Esimerkiksi piikit invariantin massan histogrammissa voivat viitata tietyn energiseen emohiukkaseen.
[Lisätietoa englanniksi](https://profmattstrassler.com/articles-and-posts/particle-physics-basics/mass-energy-matter-etc/mass-and-energy/)


<br>
 <img src="https://github.com/cms-opendata-education/cms-jupyter-materials-finnish/blob/master/Kuvat/inv_massa.PNG?raw=true"  align="left" width="600px" title="[Kuva 1]  Törmäyksestä syntyneiden kahden myonin invariantin massan jakauma vuosilta 2017 ja 2018. Jakaumasta erottaa selvästi niiden hiukkasten piikit jotka voivat hajota kahdeksi myoniksi.">
 <br>




****
### Datan hakeminen

Nyt pääsemme vihdoin käyttämään avointa dataan. Kokeillaan CMS kokeen datalla vuodelta 2011. Dataan on valikoitu tapahtumia joissa syntyy kaksi suurienergistä myonia [1]. Data on tallennettu csv muotoon tiodostoihin 'Zmumu_Run2011A.csv' ja 'Zee_Run2011A.csv' [2]. Alustetaan lasku hakemalla käyttämämme data sekä tarvittavat moduulit (_koodi kirjastot jotka mahdollistavat valmiiksi määriteltyjen funktioiden käytön_).

In [9]:
# Haetaan tarvittavat moduulit. Nimetään pandas-moduuli 'pd'ksi ja numpy-moduuli 'np'ksi.
import pandas as pd
import numpy as np

# Haetaan data ja tallennetaan se muuttujaan 'datasetti'.
datasetti = pd.read_csv('https://raw.githubusercontent.com/cms-opendata-education/cms-jupyter-materials-finnish/master/Data/Zmumu_Run2011A.csv') #('../../Data/Zmumu_Run2011A.csv')

Saamme tarkistettua 'datasettiin' tallentamamme datan sisällön tulostamalla viisi ensimmäistä tapahtumaa alla olevalla koodilla:

In [10]:
datasetti.head()

,Run,Event,pt1,eta1,phi1,Q1,dxy1,iso1,pt2,eta2,phi2,Q2,dxy2,iso2
0,165617,74969122,54.7055,-0.432396,2.574210,1,-0.074544,0.499921,34.2464,-0.988480,-0.498704,-1,0.071222,3.42214
1,165617,75138253,24.5872,-2.052200,2.866570,-1,-0.055437,0.000000,28.5389,0.385163,-1.991170,1,0.051477,0.00000
2,165617,75887636,31.7386,-2.259450,-1.332290,-1,0.087917,0.000000,30.2344,-0.468419,1.883310,1,-0.087639,0.00000
3,165617,75779415,39.7394,-0.712338,-0.312266,1,0.058481,0.000000,48.2790,-0.195625,2.970320,-1,-0.049201,0.00000
4,165617,75098104,41.2998,-0.157055,-3.040770,1,-0.030463,1.228040,43.4508,0.590958,-0.042756,-1,0.044175,0.00000


Haetussa datassa on monta eventtiä joista jokaisesta on tallennettu useita arvoja. Tehtävän kannalta oleellisia ovat,
   - __Run__ ja __Event__ : kertovat mistä törmäyksestä kyseisen rivin data on peräisin.
   - __pt__ : $p_T$ on kyseisen myonin (1 tai 2) kohtisuora liikemäärä hiukkassuihkua vastaan.
   - __eta__ : $\eta$ on pseudorapiditeetti, johon palataan tarkemmin tehtävissä 5 ja 6
   - __phi__ : $\phi$ (atsimuuttikulma) on kulma jonka suuntaa hiukkanen kulkee jos detektoria tarkastellaan hiukkassuihkun akselilta
   - __Q__ : on myonin varaus. Jokaisessa dataan valitussa törmäyksessä syntyy myoni ja antimyoni ($\mu^+ , \mu^-$) [1].
    
Huomataan ettei invarianttia massaa ole valmiiksi datassa, koska sitä ei voi mitata vaan se täytyy laskea. Se onnistuu nyt helposti.

****
### Laskun suorittaminen

Käytetään laskussa seuraavaa lauseketta invariantille massalle $M$. Lausekkeessa $p_T$ on liikemäärän hiukkassuihkua vastaan kohtisuora komponentti, $\eta$ pseudorapiditeetti ja $\phi$ atsimuuttikulma:

$$M = \sqrt{2p_{T1}p_{T2}(\cosh(\eta_1-\eta_2)-\cos(\phi_1-\phi_2))}.$$

Invariantin massan voi laskea (jokseenkin helpommin) myös kaavalla

$$ M = \sqrt{(E_1 + E_2)^2 - \|\textbf{p}_1 + \textbf{p}_2 \| ^2},$$

missä $\textbf{p}$ siis sisältää liikemäärän kaikki komponentit. Todennäköisesti törmäät tähän tapaan muissa harjoituksissa. Tällä kertaa käytetään kuitenkin ylempää kaavaa, sillä kaikki sen muuttujat löytyvät hakemastamme datatsta suoraan.

Alla olevassa laskussa käytämme apuna _numpy_-moduulia, jonka nimesimme yllä _np_:ksi. Moduulin avulla voimme suorittaa matemaattisia komentoja, esimerkiksi _sqrt_ ja _cosh_ kutsumalla ensin moduulin nimeä ja pisteellä erottaen komentoa (esimerkiksi _np.sqrt_). Koodissa esiintyvät pt1, pt2, eta1, eta2, phi1 ja phi2 viittaavat hakemamme datan suureiden nimiin. 

Nyt voimme laskea invariatin massan arvot eri tapahtumille. _Numpy_-moduuli osaa atuomaattisesti laskea arvot kaikille tapahtumille, kun annamme laskun sille muodossa, jossa esiintyy yllä mainitut viitteet suureiden nimiin. Tällöin siis lasku lasketaan jokaiselle riville erikseen.

In [11]:
invariantti_massa = np.sqrt(2*datasetti.pt1*datasetti.pt2*
                            (np.cosh(datasetti.eta1-datasetti.eta2) - np.cos(datasetti.phi1-datasetti.phi2)))

Laskun jälkeen voimme tarkastaa, minkälaisia tuloksia luomaamme muuttujaan 'invariantti_massa' tallentui tulostamalla muuttujan sisällön.

In [17]:
print(invariantti_massa)
#voit myös piirtää histogrammin poistamalla alta '#' merkit. Tähän palataan tarkemmin tehtävässä 2
#import matplotlib.pyplot as plt
#plt.hist(invariantti_massa, bins=500)


0         89.885744
1         88.810987
2         88.472502
3         90.332620
4         90.514507
5         78.860094
6         92.362439
7         63.757254
8         93.118647
9         92.941701
10        88.896587
11        85.363437
12        87.934228
13        90.414225
14        65.034823
15        89.348601
16        91.297563
17        92.258489
18        92.493002
19        68.358712
20        92.112023
21        91.171556
22        85.909734
23        91.645777
24        88.392430
25        91.760237
26        91.179805
27        91.785881
28        91.349970
29        99.112667
            ...    
10553     91.027596
10554     93.261764
10555     76.810535
10556     80.206206
10557     89.126026
10558     96.116627
10559     89.928690
10560     87.901985
10561     90.919888
10562     90.767179
10563     63.837491
10564     76.605770
10565     67.312657
10566     89.984530
10567     94.494826
10568     89.234208
10569     91.644511
10570     67.081134
10571     91.920312


Mikäli haluat lisätä lasketut arvot uutena sarakkeena tallennettuun taulukkoon, niin katso esimerkki [täältä](https://github.com/cms-opendata-education/cms-jupyter-materials-finnish/blob/master/Johdatus-jupyteriin/Lisaa-sarake-taulukkoon.ipynb).

### Lähteet


[1] Thomas McCauley (2016). Zmumu. Jupyter Notebook file. 
Url: https://github.com/tpmccauley/cmsopendata-jupyter/blob/hst-0.1/Zmumu.ipynb


[2]  CMS collaboration (2016). DoubleMu primary dataset in AOD format from RunA of 2011 (/DoubleMu/Run2011A-12Oct2013-v1/AOD). CERN Open Data Portal. DOI: [10.7483/OPENDATA.CMS.RZ34.QR6N](http://doi.org/10.7483/OPENDATA.CMS.RZ34.QR6N).


[3] Hiukkasfysiikan sanastoa englanniksi
Url: https://atlas.cern/glossary

[4] Determination of the off-shell Higgs boson signal strength in the high-mass ZZ final state with the ATLAS detector.
Url: https://atlas.cern/updates/physics-briefing/higgs-boson-s-shadow

### Kuvat
[Kuva 1] HLT Dimuon Invariant Mass Distributions in 2017 and 2018 ( CMS DP-2018/055)
Url: https://twiki.cern.ch/twiki/bin/view/CMSPublic/HLTDiMuon2017and2018